In [68]:
#!/usr/bin/python

import pickle
import cPickle
import numpy

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif



def preprocess(words_file = r"C:\Users\dehaeth\Documents\Tools\Python\Oefeningen\Udacity\Machine learning intor\ud120-projects-master\tools\word_data.pkl", authors_file=r"C:\Users\dehaeth\Documents\Tools\Python\Oefeningen\Udacity\Machine learning intor\ud120-projects-master\tools\email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "r")
    authors = pickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, "r")
    word_data = cPickle.load(words_file_handler)

    words_file_handler.close()

    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)



    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print "no. of Chris training emails:", sum(labels_train)
    print "no. of Sara training emails:", len(labels_train)-sum(labels_train)
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test


In [15]:
import sys
from time import time
sys.path.append("../tools/")

import time


In [16]:
features_train, features_test, labels_train,labels_test = preprocess()

no. of Chris training emails: 7936
no. of Sara training emails: 7884


In [17]:
from sklearn.naive_bayes import GaussianNB
t0 = time.time()
nb = GaussianNB()
nb.fit(features_train,labels_train)
labels_predict = nb.predict(features_test)
from sklearn.metrics import accuracy_score
print accuracy_score(labels_test,labels_predict)
t1 = time.time()
print t1 - t0

0.973265073948
1.11100006104


In [18]:
from sklearn.svm import SVC
t2 = time.time()
svm = SVC(kernel="linear")
svm.fit(features_train,labels_train)
labels_predict = svm.predict(features_test)
from sklearn.metrics import accuracy_score
print accuracy_score(labels_test,labels_predict)
t3 = time.time()
print t3-t2

0.984072810011
206.618999958


#### EFFECT OF SMALLER SETS

In [19]:
from sklearn.naive_bayes import GaussianNB
t0 = time.time()
nb = GaussianNB()
features_train_sub = features_train[:len(features_train)/100]
labels_train_sub = labels_train[:len(labels_train)/100]
nb.fit(features_train_sub,labels_train_sub)
labels_predict = nb.predict(features_test)
from sklearn.metrics import accuracy_score
print accuracy_score(labels_test,labels_predict)
t1 = time.time()
print t1 - t0

0.919226393629
0.163000106812


In [20]:
from sklearn.svm import SVC
t2 = time.time()
svm = SVC(kernel="linear")
features_train_sub = features_train[:len(features_train)/100]
labels_train_sub = labels_train[:len(labels_train)/100]
svm.fit(features_train_sub,labels_train_sub)
labels_predict = svm.predict(features_test)
from sklearn.metrics import accuracy_score
print accuracy_score(labels_test,labels_predict)
t3 = time.time()
print t3-t2

0.884527872582
1.23699998856


#### Effect of rbf Kernell

In [21]:
from sklearn.svm import SVC
t2 = time.time()
svm = SVC(kernel="rbf")
features_train_sub = features_train[:len(features_train)/100]
labels_train_sub = labels_train[:len(labels_train)/100]
svm.fit(features_train_sub,labels_train_sub)
labels_predict = svm.predict(features_test)
from sklearn.metrics import accuracy_score
print accuracy_score(labels_test,labels_predict)
t3 = time.time()
print t3-t2

0.616040955631
1.41700005531


#### Effect of C value

In [38]:
from sklearn.svm import SVC
t2 = time.time()
svm = SVC(kernel="rbf", C = 5700)
features_train_sub = features_train[:len(features_train)/100]
labels_train_sub = labels_train[:len(labels_train)/100]
svm.fit(features_train_sub,labels_train_sub)
labels_predict = svm.predict(features_test)
from sklearn.metrics import accuracy_score
print accuracy_score(labels_test,labels_predict)
t3 = time.time()


0.900455062571


#### With optimized parameters

In [39]:
from sklearn.svm import SVC
t2 = time.time()
svm = SVC(kernel="rbf", C = 5700)
svm.fit(features_train,labels_train)
labels_predict = svm.predict(features_test)
from sklearn.metrics import accuracy_score
print accuracy_score(labels_test,labels_predict)
t3 = time.time()

0.990898748578


#### Specific prediction

In [46]:
svm = SVC(kernel="rbf", C = 10000)
features_train_sub = features_train[:len(features_train)/100]
labels_train_sub = labels_train[:len(labels_train)/100]
svm.fit(features_train_sub,labels_train_sub)
labels_predict = svm.predict(features_test)
from sklearn.metrics import accuracy_score
print labels_predict[50]

1


#### Number of 1 predictions

In [48]:
import numpy as np
svm = SVC(kernel="rbf", C = 10000)
svm.fit(features_train,labels_train)
labels_predict = svm.predict(features_test)
from sklearn.metrics import accuracy_score
print np.sum(labels_predict)

877


## Decision trees

### basic setup

In [52]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(min_samples_split = 40)

### training first model

In [69]:
features_train, features_test, labels_train,labels_test = preprocess()
dt.fit(features_train,labels_train)

no. of Chris training emails: 7936
no. of Sara training emails: 7884


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=40, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [71]:
labels_predict_dt = dt.predict(features_test)
print accuracy_score(labels_test,labels_predict_dt)

0.977815699659


### number of features

In [61]:
features_train, features_test, labels_train,labels_test = preprocess()
print len(features_train[0])

no. of Chris training emails: 7936
no. of Sara training emails: 7884
379


## K Nearest Neighbors

### basic setup

In [89]:
from sklearn.neighbors import KNeighborsClassifier

features_train_sub = features_train[:len(features_train)/10]
labels_train_sub = labels_train[:len(labels_train)/10]

kn = KNeighborsClassifier(3, weights="uniform")
kn.fit(features_train_sub,labels_train_sub)
labels_predict_kn = kn.predict(features_test)
print accuracy_score(labels_test,labels_predict_kn)

0.627986348123


### playing with weights

In [90]:
kn = KNeighborsClassifier(3, weights="distance")
kn.fit(features_train_sub,labels_train_sub)
labels_predict_kn = kn.predict(features_test)
print accuracy_score(labels_test,labels_predict_kn)

0.669510807736


### playing with k

In [94]:
kn = KNeighborsClassifier(1, weights="distance")
kn.fit(features_train_sub,labels_train_sub)
labels_predict_kn = kn.predict(features_test)
print accuracy_score(labels_test,labels_predict_kn)

0.740045506257


### full set

In [95]:
kn = KNeighborsClassifier(1, weights="distance")
kn.fit(features_train,labels_train)
labels_predict_kn = kn.predict(features_test)
print accuracy_score(labels_test,labels_predict_kn)

0.978953356086
